In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_path = '/content/drive/My Drive/CBD Robotics course/Assignment 08/fruits/fruits-360/Training/'
test_path = '/content/drive/My Drive/CBD Robotics course/Assignment 08/fruits/fruits-360/Test/'

In [0]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:

# create a data generator
datagen = ImageDataGenerator(rescale=1./255)
# load and iterate test datas
test_it = datagen.flow_from_directory(directory=test_path, target_size=(100, 100), color_mode="rgb", class_mode='categorical', batch_size=64)

Found 19053 images belonging to 111 classes.


In [0]:
# create a data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# load and iterate training dataset
train_it = datagen.flow_from_directory(directory=train_path, target_size=(100, 100), color_mode="rgb",  class_mode='categorical', batch_size=64, subset='training')
val_it = datagen.flow_from_directory(directory=train_path, target_size=(100, 100), color_mode="rgb",  class_mode='categorical', batch_size=64, subset='validation')

Found 45454 images belonging to 111 classes.
Found 11327 images belonging to 111 classes.


Những nhãn trên tập train là:

In [0]:
train_label = list(train_it.class_indices.keys())
print(train_label)
print('Có số lượng:', len(train_label))

['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Blueberry', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos', 'Dates', 'Granadilla', 'Grape Blue', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grape White 3', 'Grape White 4', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Hazelnut', 'Huckleberry', 'Kaki', 'Kiwi', 'Kohlrabi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Mango Red', 'Mangostan', 'Maracuja', 'Melon Piel de Sapo', 'Mulberry', 'Nectarine', 'Nectarine Flat', 'Onion Red', 'Onion Red Peeled', 'Onion White', 'Orange', 

In [0]:
test_label = list(test_it.class_indices.keys())
print(test_label)
print('Có số lượng:', len(test_label))

['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Blueberry', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos', 'Dates', 'Granadilla', 'Grape Blue', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grape White 3', 'Grape White 4', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Hazelnut', 'Huckleberry', 'Kaki', 'Kiwi', 'Kohlrabi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Mango Red', 'Mangostan', 'Maracuja', 'Melon Piel de Sapo', 'Mulberry', 'Nectarine', 'Nectarine Flat', 'Onion Red', 'Onion Red Peeled', 'Onion White', 'Orange', 

In [0]:
# Số lượng thực thể trên tập train, val và test
print('Số thực thể trên train:', train_it.samples)
print('Số thực thể trên train:', val_it.samples)
print('Số thực thể trên train:', test_it.samples)

Số thực thể trên train: 45454
Số thực thể trên train: 11327
Số thực thể trên train: 19053


In [0]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras import backend as K
#K.set_image_dim_ordering('tf')

In [0]:
type(train_it)

keras_preprocessing.image.directory_iterator.DirectoryIterator

In [0]:
# Saving Checkpoints using Callbacks in Keras
from keras.callbacks import *
filepath='/content/drive/My Drive/CBD Robotics course/Assignment 08/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# create larger_model
def larger_model():
  # create model
  model = Sequential()
  model.add(Conv2D(32, (3,3), input_shape=(100, 100, 3), activation='relu', padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(111, activation='softmax'))
  # compile model
  lrate= 0.01
  decay = lrate/25
  sgd = SGD(lr=lrate, momentum=0.9, decay = decay, nesterov=False)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

model = larger_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 50, 50, 64)        36928     
__________

In [0]:
model.load_weights('/content/drive/My Drive/CBD Robotics course/Assignment 08/epochs:001-val_acc:0.540.hdf5')

In [0]:
# fit model
batch_size = 100
model.fit_generator(
    train_it,
    steps_per_epoch = train_it.samples // batch_size,
    validation_data = val_it, 
    validation_steps = val_it.samples // batch_size,
    callbacks=callbacks_list,
    epochs = 7)

Epoch 1/7
454/454 [==============================] - 1665s 4s/step - loss: 0.6226 - acc: 0.8014 - val_loss: 0.6785 - val_acc: 0.8039

Epoch 00001: val_acc improved from 0.53996 to 0.80390, saving model to /content/drive/My Drive/CBD Robotics course/Assignment 08/epochs:001-val_acc:0.804.hdf5
Epoch 2/7
454/454 [==============================] - 65s 143ms/step - loss: 0.1615 - acc: 0.9464 - val_loss: 0.3921 - val_acc: 0.9104

Epoch 00002: val_acc improved from 0.80390 to 0.91040, saving model to /content/drive/My Drive/CBD Robotics course/Assignment 08/epochs:002-val_acc:0.910.hdf5
Epoch 3/7
454/454 [==============================] - 63s 140ms/step - loss: 0.0617 - acc: 0.9807 - val_loss: 0.3628 - val_acc: 0.9079

Epoch 00003: val_acc did not improve from 0.91040
Epoch 4/7
454/454 [==============================] - 62s 136ms/step - loss: 0.0350 - acc: 0.9882 - val_loss: 0.4741 - val_acc: 0.8968

Epoch 00004: val_acc did not improve from 0.91040
Epoch 5/7
454/454 [========================

In [0]:
# evaluate model
loss = model.evaluate_generator(test_it, steps=24)
print('Accuracy: %.2f%%' % (loss[1]*100))

Accuracy: 93.82%


In [0]:
model.save('/content/drive/My Drive/CBD Robotics course/Assignment 08/trained_model1.h5')

**Note: Load lại model với 5 epochs để so sánh**

In [0]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/CBD Robotics course/Assignment 08/trained_model.h5')

In [0]:
# evaluate model
loss = model.evaluate_generator(test_it, steps=24)
print('Accuracy: %.2f%%' % (loss[1]*100))

Accuracy: 92.90%


In [0]:
# evaluate model
loss = model.evaluate_generator(test_it, steps=64)
print('Accuracy: %.2f%%' % (loss[1]*100))

Accuracy: 93.43%


In [0]:
# evaluate model
loss = model.evaluate_generator(test_it, steps= test_it.samples // batch_size)
print('Accuracy: %.2f%%' % (loss[1]*100))

Accuracy: 93.28%
